# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition.

# 1) Importing data

In [183]:
import numpy as np 
import pandas as pd

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [184]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = 'subtask4b_collection_data.pkl' #MODIFY PATH

In [185]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [186]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [187]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [188]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_QUERY_TRAIN_DATA = 'subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = 'subtask4b_query_tweets_dev.tsv' #MODIFY PATH

In [189]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [190]:
df_query_train.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [191]:
df_query_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     12853 non-null  int64 
 1   tweet_text  12853 non-null  object
 2   cord_uid    12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [192]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [193]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


# 2) Running the baseline
The following code runs a BM25 baseline.


In [194]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi


In [195]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [196]:
def get_top_cord_uids(query, top_k=5):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:top_k]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [197]:
# Retrieve topk candidates using the BM25 model
df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x, top_k=5))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x, top_k=5))

# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [198]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


In [199]:
results_train_BL = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev_BL = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(f"Results on the train set: {results_train_BL}")
print(f"Results on the dev set: {results_dev_BL}")

Results on the train set: {1: 0.5079747918773827, 5: 0.5508999196037242, 10: 0.5508999196037242}
Results on the dev set: {1: 0.505, 5: 0.5520357142857142, 10: 0.5520357142857142}


# 4) Exporting results to prepare the submission on Codalab

In [200]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [201]:
df_query_dev[['post_id', 'preds']].to_csv('predictions_BL.tsv', index=None, sep='\t')

# 5) Task Implementation: Traditional IR Models

## 5.1) TF-IDF

In [ ]:
# if necessary
!pip install collections
!pip install sklearn
!pip install spacy
!pip install nltk

In [202]:
from collections import defaultdict, Counter
from sklearn.preprocessing import normalize

df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [203]:
def preprocess(text):
    return text.lower().split()

doc_tokens = []
doc_freq = defaultdict(int)

for idx, row in df_collection.iterrows():
    text = f"{row['title']} {row['abstract']}"
    tokens = preprocess(text)
    doc_tokens.append(tokens)
    tf = Counter(tokens)
    for term in tf.keys():
        doc_freq[term] += 1

vocab = sorted(doc_freq.keys())


In [204]:
# sparse TF-IDF matrix
docs_text = [' '.join(tokens) for tokens in doc_tokens]
vectorizer = TfidfVectorizer(vocabulary=vocab)
tfidf_sparse = vectorizer.fit_transform(docs_text)  

# normalization
tfidf_sparse = normalize(tfidf_sparse, norm='l2', axis=1)

In [205]:
# preprocess queries like docs
queries = df_query_train['tweet_text'].tolist() + df_query_dev['tweet_text'].tolist()
queries_tokens = [preprocess(q) for q in queries]
queries_text = [' '.join(tokens) for tokens in queries_tokens]

# queries into TF-IDF
query_tfidf = vectorizer.transform(queries_text) 

In [206]:
# cosine similarity matrix
similarity = query_tfidf.dot(tfidf_sparse.T)


In [207]:
# top-k docs for each query
top_k = 5

topk_indices_unsorted = np.argpartition(-similarity.toarray(), top_k-1, axis=1)[:, :top_k]
topk_indices_sorted = np.zeros_like(topk_indices_unsorted)

for i, inds in enumerate(topk_indices_unsorted):
    sim_scores = similarity[i, inds].toarray().flatten()
    sorted_order = np.argsort(-sim_scores)
    topk_indices_sorted[i] = inds[sorted_order]


doc_cord_uids = df_collection['cord_uid'].values

In [208]:
# scoring on train and dev sets
train_len = len(df_query_train)
train_topk = [[doc_cord_uids[idx] for idx in row] for row in topk_indices_sorted[:train_len]]
dev_topk = [[doc_cord_uids[idx] for idx in row] for row in topk_indices_sorted[train_len:]]

df_query_train['tfidf_topk'] = train_topk
df_query_dev['tfidf_topk'] = dev_topk

In [209]:
# evaluate
results_train_tfidf = get_performance_mrr(df_query_train, 'cord_uid', 'tfidf_topk')
results_dev_tfidf = get_performance_mrr(df_query_dev, 'cord_uid', 'tfidf_topk')

print(f"TF-IDF Results on the train set: {results_train_tfidf}")
print(f"TF-IDF Results on the dev set: {results_dev_tfidf}")

TF-IDF Results on the train set: {1: 0.47918773827122074, 5: 0.5422015612438081, 10: 0.5422015612438081}
TF-IDF Results on the dev set: {1: 0.4764285714285714, 5: 0.5388452380952382, 10: 0.5388452380952382}


In [210]:
# exporting
df_query_dev['preds'] = df_query_dev['tfidf_topk'].apply(lambda x: x[:5])
df_query_dev[['post_id', 'preds']].to_csv('tfidf_predictions.tsv', index=None, sep='\t')

## 5.2) TF-IDF with Lemmatization


In [211]:
import spacy
nlp = spacy.load('en_core_web_sm')
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [212]:
def preprocess_L(text):
    doc = nlp(text.lower())
    return [token.lemma_ for token in doc if not token.is_punct and not token.is_space]

doc_tokens_L = []
doc_freq_L = defaultdict(int)

for idx, row in df_collection.iterrows():
    text = f"{row['title']} {row['abstract']}"
    tokens = preprocess_L(text)
    doc_tokens_L.append(tokens)
    tf = Counter(tokens)
    for term in tf.keys():
        doc_freq_L[term] += 1

vocab_L = sorted(doc_freq_L.keys())

In [213]:
# sparse TF-IDF matrix
docs_text_L = [' '.join(tokens) for tokens in doc_tokens_L]
vectorizer_L = TfidfVectorizer(vocabulary=vocab_L)
tfidf_sparse_L = vectorizer_L.fit_transform(docs_text_L)

# normalization
tfidf_sparse_L = normalize(tfidf_sparse_L, norm='l2', axis=1)

/Users/jacopo/opt/miniconda3/envs/air_env2/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [214]:
# preprocess queries like docs
queries_L = df_query_train['tweet_text'].tolist() + df_query_dev['tweet_text'].tolist()
queries_tokens_L = [preprocess_L(q) for q in queries_L]
queries_text_L = [' '.join(tokens) for tokens in queries_tokens_L]

# queries into TF-IDF
query_tfidf_L = vectorizer_L.transform(queries_text_L)


In [215]:
# cosine similarity matrix
similarity_L = query_tfidf_L.dot(tfidf_sparse_L.T)

In [216]:
# top-k docs for each query
top_k = 5

topk_indices_unsorted_L = np.argpartition(-similarity_L.toarray(), top_k-1, axis=1)[:, :top_k]
topk_indices_sorted_L = np.zeros_like(topk_indices_unsorted_L)

for i, inds in enumerate(topk_indices_unsorted_L):
    sim_scores_L = similarity_L[i, inds].toarray().flatten()
    sorted_order_L = np.argsort(-sim_scores_L)
    topk_indices_sorted_L[i] = inds[sorted_order_L]


doc_cord_uids_L = df_collection['cord_uid'].values

In [217]:
# scoring on train and dev sets
train_len = len(df_query_train)
train_topk_L = [[doc_cord_uids_L[idx] for idx in row] for row in topk_indices_sorted_L[:train_len]]
dev_topk_L = [[doc_cord_uids_L[idx] for idx in row] for row in topk_indices_sorted_L[train_len:]]

df_query_train['tfidf_topk'] = train_topk_L
df_query_dev['tfidf_topk'] = dev_topk_L

In [218]:
# evaluate
results_train_tfidf_L = get_performance_mrr(df_query_train, 'cord_uid', 'tfidf_topk')
results_dev_tfidf_L = get_performance_mrr(df_query_dev, 'cord_uid', 'tfidf_topk')

print(f"TF-IDF Results on the train set: {results_train_tfidf_L}")
print(f"TF-IDF Results on the dev set: {results_dev_tfidf_L}")

TF-IDF Results on the train set: {1: 0.47179646775071965, 5: 0.5382297258746337, 10: 0.5382297258746337}
TF-IDF Results on the dev set: {1: 0.4742857142857143, 5: 0.5370238095238095, 10: 0.5370238095238095}


In [219]:
# exporting
df_query_dev['preds'] = df_query_dev['tfidf_topk'].apply(lambda x: x[:5])
df_query_dev[['post_id', 'preds']].to_csv('tfidf_predictions_L.tsv', index=None, sep='\t')

## 5.3) TF-IDF with Lemmatization, Bigrams, and Stop Words

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from nltk.util import ngrams

nlp = spacy.load('en_core_web_sm')
stop_words = set(ENGLISH_STOP_WORDS)

df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [221]:
def preprocess_LBSW(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc 
              if not token.is_punct and not token.is_space and token.lemma_ not in stop_words]
    
    bigrams = ['_'.join(pair) for pair in ngrams(tokens, 2)]
    return tokens + bigrams


doc_tokens_LBSW = []
doc_freq_LBSW = defaultdict(int)

for idx, row in df_collection.iterrows():
    text = f"{row['title']} {row['abstract']}"
    tokens = preprocess_LBSW(text)
    doc_tokens_LBSW.append(tokens)
    
    unique_terms = set(tokens)  # Count each term only once per doc
    for term in unique_terms:
        doc_freq_LBSW[term] += 1

        
vocab_LBSW = sorted(doc_freq_LBSW.keys())


In [222]:
# sparse TF-IDF matrix
docs_text_LBSW = [' '.join(tokens) for tokens in doc_tokens_LBSW]
vectorizer_LBSW = TfidfVectorizer(vocabulary=vocab_LBSW)
tfidf_sparse_LBSW = vectorizer_LBSW.fit_transform(docs_text_LBSW)

# normalization
tfidf_sparse_LBSW = normalize(tfidf_sparse_LBSW, norm='l2', axis=1)


/Users/jacopo/opt/miniconda3/envs/air_env2/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1381: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [223]:
# preprocess queries like docs
queries_LBSW = df_query_train['tweet_text'].tolist() + df_query_dev['tweet_text'].tolist()
queries_tokens_LBSW = [preprocess_LBSW(q) for q in queries_LBSW]
queries_text_LBSW = [' '.join(tokens) for tokens in queries_tokens_LBSW]

# queries into TF-IDF
query_tfidf_LBSW = vectorizer_LBSW.transform(queries_text_LBSW)

In [224]:
# cosine similarity matrix
similarity_LBSW = query_tfidf_LBSW.dot(tfidf_sparse_LBSW.T)

In [225]:
# top-k docs for each query
top_k = 5

topk_indices_unsorted_LBSW = np.argpartition(-similarity_LBSW.toarray(), top_k-1, axis=1)[:, :top_k]
topk_indices_sorted_LBSW = np.zeros_like(topk_indices_unsorted_LBSW)

for i, inds in enumerate(topk_indices_unsorted_LBSW):
    sim_scores_LBSW = similarity_LBSW[i, inds].toarray().flatten()
    sorted_order_LBSW = np.argsort(-sim_scores_LBSW)
    topk_indices_sorted_LBSW[i] = inds[sorted_order_LBSW]

doc_cord_uids_LBSW = df_collection['cord_uid'].values

In [226]:
# scoring on train and dev sets
train_len = len(df_query_train)
train_topk_LBSW = [[doc_cord_uids_LBSW[idx] for idx in row] for row in topk_indices_sorted_LBSW[:train_len]]
dev_topk_LBSW = [[doc_cord_uids_LBSW[idx] for idx in row] for row in topk_indices_sorted_LBSW[train_len:]]

df_query_train['tfidf_topk'] = train_topk_LBSW
df_query_dev['tfidf_topk'] = dev_topk_LBSW

In [227]:
# evaluate
results_train_tfidf_LBSW = get_performance_mrr(df_query_train, 'cord_uid', 'tfidf_topk')
results_dev_tfidf_LBSW = get_performance_mrr(df_query_dev, 'cord_uid', 'tfidf_topk')

print(f"TF-IDF Results on the train set: {results_train_tfidf_LBSW}")
print(f"TF-IDF Results on the dev set: {results_dev_tfidf_LBSW}")

TF-IDF Results on the train set: {1: 0.5180891620633316, 5: 0.5755167405793719, 10: 0.5755167405793719}
TF-IDF Results on the dev set: {1: 0.5114285714285715, 5: 0.5674642857142856, 10: 0.5674642857142856}


In [228]:
# exporting
df_query_dev['preds'] = df_query_dev['tfidf_topk'].apply(lambda x: x[:5])
df_query_dev[['post_id', 'preds']].to_csv('tfidf_predictions_LBSW.tsv', index=None, sep='\t')

In [229]:
print(f"Results on the dev set (BL): {results_dev_BL}")
print(f"Results on the dev set (TF-IDF): {results_dev_tfidf}")
print(f"Results on the dev set (L): {results_dev_tfidf_L}")
print(f"Results on the dev set (LBSW): {results_dev_tfidf_LBSW}")


Results on the dev set (BL): {1: 0.505, 5: 0.5520357142857142, 10: 0.5520357142857142}
Results on the dev set (TF-IDF): {1: 0.4764285714285714, 5: 0.5388452380952382, 10: 0.5388452380952382}
Results on the dev set (L): {1: 0.4742857142857143, 5: 0.5370238095238095, 10: 0.5370238095238095}
Results on the dev set (LBSW): {1: 0.5114285714285715, 5: 0.5674642857142856, 10: 0.5674642857142856}


# 6) TF-IDF with Lemmatization, Bigrams, and Stop Words on Test Set

In [230]:
df_query_test = pd.read_csv('subtask4b_query_tweets_test.tsv', sep='\t')


In [231]:
queries_test = df_query_test['tweet_text'].tolist()
queries_tokens_test_LBSW = [preprocess_LBSW(q) for q in queries_test]
queries_text_test_LBSW = [' '.join(tokens) for tokens in queries_tokens_test_LBSW]


In [232]:
query_tfidf_test_LBSW = vectorizer_LBSW.transform(queries_text_test_LBSW)

In [233]:
similarity_test_LBSW = query_tfidf_test_LBSW.dot(tfidf_sparse_LBSW.T)

In [234]:
top_k = 5
topk_indices_unsorted_test_LBSW = np.argpartition(-similarity_test_LBSW.toarray(), top_k-1, axis=1)[:, :top_k]

topk_indices_sorted_test_LBSW = np.zeros_like(topk_indices_unsorted_test_LBSW)

for i, inds in enumerate(topk_indices_unsorted_test_LBSW):
    sim_scores = similarity_test_LBSW[i, inds].toarray().flatten()
    sorted_order = np.argsort(-sim_scores)
    topk_indices_sorted_test_LBSW[i] = inds[sorted_order]

In [235]:
test_topk_LBSW = [[doc_cord_uids_LBSW[idx] for idx in row] for row in topk_indices_sorted_test_LBSW]
df_query_test['tfidf_topk'] = test_topk_LBSW

In [236]:
df_query_test['preds'] = df_query_test['tfidf_topk'].apply(lambda x: x[:5])
df_query_test[['post_id', 'preds']].to_csv('tfidf_predictions_test.tsv', index=None, sep='\t')